In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time
import pandas as pd


servico = Service(ChromeDriverManager().install())

driver = webdriver.Chrome(service=servico)

#acessar site
driver.get("https://www2.aneel.gov.br/aplicacoes_liferay/tarifa/")
time.sleep(2)


#Seleciona Categoria do Agente Todos
select_cat_agente = Select(driver.find_element(By.NAME,"CategoriaAgente"))
select_cat_agente.select_by_visible_text('Todos')
time.sleep(1)

select_agente = Select(driver.find_element(By.NAME,"Agentes"))
#Traz todas as opcoes de Agente
options_agente = [x.text for x in select_agente.options]
i=2
j=0

for element in options_agente[2:]:
    #Seleciona o Agente
    #select_agente.select_by_index(element)
    #select_agente[element]
    select_agente.select_by_index(i)
    i+=1
    time.sleep(1)

    try:
        #Seleciona Tipo de Processo
        select_tipo_processo = Select(driver.find_element(By.NAME,"TipoProcesso"))
        select_tipo_processo.select_by_visible_text('Todos')
        time.sleep(1)

        #Seleciona Ano
        select_ano = Select(driver.find_element(By.NAME,"Ano"))
        select_ano.select_by_visible_text('Todos')
        time.sleep(1)

        #Clica em Procurar
        driver.find_element(By.XPATH,'/html/body/table/tbody/tr[2]/td[5]/input').click()
        time.sleep(1)

        #Pegar o link de download do arquivo PCAT mais atualizado (estrutura tarifaria)
        link = driver.find_element(By.XPATH, '/html/body/div/table/tbody/tr[4]/td[7]/a').get_attribute('href')
    except:
        continue

    try:
        planilha = pd.ExcelFile(link, engine = 'openpyxl')
        df = pd.read_excel(planilha, 'TA - Aplicação', skiprows = 2)
        print("Tabela com TA Aplicação:", element)
        print(f'Tabela {i-2} de {len(options_agente)-2}')
    except:
        print("Tabela sem TA Aplicação:", element)
        print(f'Tabela {i-2} de {len(options_agente)-2}')
        continue
    
    #limpeza
    df.drop('Unnamed: 0', inplace=True, axis=1)

    #adiciona a coluna com o nome da concessionaria
    df.insert(0,'CONCESSIONARIA', element)

    #filtra apenas as colunas de interesse
    df2 = df.filter(items=['CONCESSIONARIA', 'SUBGRUPO', 'MODALIDADE', 'CLASSE', 'SUBCLASSE', 'DETALHE', 'POSTO', 'UNIDADE', 'ACESSANTE','Total TE', 'Total TUSD', 'TUSD_FioB'])
    
    #concatena todas os DF
    if j == 0:
        df_geral = df2
        j = 1
    else:
        df_geral = pd.concat([df_geral, df2], ignore_index=True)

df_geral.to_csv('tarifas_direto.csv', index=False)
df_geral.to_json('tarifas.json', orient='index')


driver.close()
print('Extração Concluída')


Tabela com TA Aplicação: AME
Tabela 1 de 120
Tabela com TA Aplicação: Boa Vista
Tabela 2 de 120
Tabela com TA Aplicação: CASTRO - DIS
Tabela 4 de 120
Tabela com TA Aplicação: CEA
Tabela 5 de 120
Tabela com TA Aplicação: CEB
Tabela 6 de 120
Tabela com TA Aplicação: CEBDIS
Tabela 7 de 120
Tabela com TA Aplicação: CEDRAP
Tabela 8 de 120
Tabela com TA Aplicação: CEDRI
Tabela 9 de 120
Tabela com TA Aplicação: CEEE-D
Tabela 10 de 120
Tabela com TA Aplicação: CEGERO
Tabela 11 de 120
Tabela com TA Aplicação: CELESC-DIS
Tabela 14 de 120
Tabela com TA Aplicação: CELETRO
Tabela 15 de 120
Tabela com TA Aplicação: CELG-D
Tabela 17 de 120
Tabela com TA Aplicação: CELPE
Tabela 18 de 120
Tabela com TA Aplicação: CEMIG-D
Tabela 19 de 120
Tabela com TA Aplicação: CEMIRIM
Tabela 20 de 120
Tabela com TA Aplicação: CEPRAG
Tabela 21 de 120
Tabela sem TA Aplicação: CERAÇÁ
Tabela com TA Aplicação: CERAL ANITÁPOLIS
Tabela 23 de 120
Tabela com TA Aplicação: CERAL ARAPOTI
Tabela 24 de 120
Tabela com TA Aplicação

In [9]:
df_geral

,CONCESSIONARIA,SUBGRUPO,MODALIDADE,CLASSE,SUBCLASSE,DETALHE,POSTO,UNIDADE,ACESSANTE,Total TE,Total TUSD,TUSD_FioB
0,AME,A3,Azul,Não se aplica,Não se aplica,APE,Ponta,MWh,Não se aplica,0.00,31.69,0.000000
1,AME,A3,Azul,Não se aplica,Não se aplica,APE,Ponta,kW,Não se aplica,0.00,8.74,4.185401
2,AME,A3,Azul,Não se aplica,Não se aplica,APE,Fora ponta,MWh,Não se aplica,0.00,31.69,0.000000
3,AME,A3,Azul,Não se aplica,Não se aplica,APE,Fora ponta,kW,Não se aplica,0.00,7.12,3.402224
4,AME,A3,Azul,Não se aplica,Não se aplica,Não se aplica,Ponta,MWh,Não se aplica,456.17,84.42,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
97,Boa Vista,B3,Branca,Não se aplica,Não se aplica,Não se aplica,Fora ponta,MWh,Não se aplica,317.39,336.05,185.802922
98,Boa Vista,B3,Convencional,Não se aplica,Não se aplica,Não se aplica,Não se aplica,MWh,Não se aplica,336.18,398.95,247.737150
99,Boa Vista,B3,Convencional pré-pagamento,Não se aplica,Não se aplica,Não se aplica,Não se aplica,MWh,Não se aplica,336.18,398.95,247.737150
100,Boa Vista,B4,Convencional,Iluminação pública,Iluminação pública – B4a,Não se aplica,Não se aplica,MWh,Não se aplica,184.90,219.42,136.255432
